In [103]:
# output of tool D
parent_folder = '/Users/kavyasrinet/Desktop/other_actions/0/'
folder_name_D = parent_folder + 'toolD/'
tool_D_out_file = folder_name_D + 'all_agreements.txt'

In [149]:
# output of tool C
parent_folder= '/Users/kavyasrinet/Desktop/other_actions/5/'
folder_name_C = parent_folder  + 'toolC/'
tool_C_out_file = folder_name_C + 'all_agreements.txt'

In [150]:
# combine outputs
# check if all keys of tool C annotated yes -> put directly
# if no , check child in t2 and combine
# construct mape of tool 1

toolC_map = {}

import ast
import os.path
from os import path
if path.exists(tool_C_out_file):
    with open(tool_C_out_file) as f:
        for line in f.readlines():
            line = line.strip()
            cmd, ref_obj_text, a_d = line.split("\t")
            if cmd in toolC_map:
                toolC_map[cmd].update(ast.literal_eval(a_d))
            else:
                toolC_map[cmd]= ast.literal_eval(a_d)
print(len(toolC_map.keys()))

27


In [151]:
# construct map of tool 2
toolD_map = {}

if path.exists(tool_D_out_file):
    with open(tool_D_out_file) as f2:
        for line in f2.readlines():
            line = line.strip()
            cmd, comparison_text, comparison_dict = line.split("\t")
            if cmd in toolD_map:
                print("BUGGGGG")
            # add the comparison dict to command -> dict
            toolD_map[cmd] = ast.literal_eval(comparison_dict)      
print(len(toolD_map.keys()))

2


In [152]:
import ast
def all_yes(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    for k, val in a_dict.items():
        if type(val) == list and val[0] == "no":
            return False
    return True

def clean_up_dict(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    new_d = {}
    for k, val in a_dict.items():
        if type(val) == list:
            if val[0] in ["yes", "no"]:
                new_d[k] = val[1]                
        elif type(val) == dict:
            new_d[k] = clean_up_dict(val)
        else:
            new_d[k] = val
    return new_d

In [153]:
# post_process spans and "contains_coreference" : "no"
def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        a = min(a, indices[i][0])
        b = max(b, indices[i][1])
    return [a, b]


def fix_spans(d):
    new_d = {}
    if type(d) == str:
        d = ast.literal_eval(d)
    for k, v in d.items():
        if k == "contains_coreference" and v == "no":
            continue
        if type(v) == list:
            new_d[k] = [0, merge_indices(v)] 
            continue
        elif type(v) == dict:
            new_d[k] = fix_spans(v)
            continue
        else:
            new_d[k] = v
    return new_d

In [154]:
def fix_ref_obj(clean_dict):
    val = clean_dict
    new_clean_dict = {}
    if 'special_reference' in val:
        new_clean_dict['special_reference'] = val['special_reference']
        val.pop('special_reference')
    if 'repeat' in val:
        new_clean_dict['repeat'] = val['repeat']
        val.pop('repeat')
    if val:
        new_clean_dict['filters'] = val
    return new_clean_dict

In [155]:
# combine and write output to a file
i  =0
# what these action will look like in the map
import json

toolC_updated_map = {}

from pprint import pprint
# update dict of toolC with tool D and keep that in tool C's map
#pprint(toolC_map)
for cmd, a_dict in toolC_map.items():
    # remove the ['yes', val] etc
    for key in a_dict.keys():
        a_dict_child = a_dict[key]
        #pprint(a_dict_child)
        clean_dict = clean_up_dict(a_dict_child)
        # add in filters to reference objects
#         val = clean_dict
#         new_clean_dict = {}
#         if 'special_reference' in val:
#             new_clean_dict['special_reference'] = val['special_reference']
#             val.pop('special_reference')
#         if 'repeat' in val:
#             new_clean_dict['repeat'] = val['repeat']
#             val.pop('repeat')
#         if val:
#             new_clean_dict['filters'] = val
        # fix reference object inside location of reference object
        if 'location' in clean_dict and 'reference_object' in clean_dict['location']:
            value =  clean_dict['location']['reference_object']
            clean_dict['location']['reference_object'] =  fix_ref_obj(value)
        new_clean_dict = fix_ref_obj(clean_dict)
        
        if all_yes(a_dict_child):
            if cmd in toolC_updated_map:
                toolC_updated_map[cmd][key] = new_clean_dict
            else:
                toolC_updated_map[cmd]= {key: new_clean_dict}
            continue
        new_clean_dict.pop('comparison', None)
        comparison_dict = toolD_map[cmd] # check on this again

        valid_dict = {}
        valid_dict[key] = {}
        #valid_dict['reference_object'] = {}
#         valid_dict['reference_object']['filters'] = clean_dict
#         valid_dict['reference_object']['filters'].update(comparison_dict)
        valid_dict[key]['filters'] = new_clean_dict
        valid_dict[key]['filters'].update(comparison_dict)
        toolC_updated_map[cmd] = valid_dict # only gets populated if filters exist
pprint(toolC_updated_map)

{'and add it to the tree': {'reference_object': {'filters': {'contains_coreference': 'yes'}}},
 'bot bring me my red pants suit': {'receiver_reference_object': {'special_reference': 'SPEAKER'},
                                    'reference_object': {'filters': {'has_colour': [[4,
                                                                                     4]],
                                                                     'has_name': [[5,
                                                                                   5],
                                                                                  [6,
                                                                                   6]]}}},
 'chop a tree': {'reference_object': {'filters': {'has_name': [[2, 2]]}}},
 'collect the mushrooms here': {'reference_object': {'filters': {'has_name': [[2,
                                                                               2]]}}},
 'find a villiage': {'reference_ob

In [156]:
print(len(toolC_updated_map.keys()))
print(len(toolC_map.keys()))

27
27


In [157]:
# output of tool 1
folder_name_A = parent_folder + 'toolA/'
tool_A_out_file = folder_name_A + 'all_agreements.txt'

In [158]:
# output of tool 2
folder_name_B = parent_folder + 'toolB/'
tool_B_out_file = folder_name_B + 'all_agreements.txt'

In [159]:
# combine outputs
# check if all keys of t1 annotated yes -> put directly
# if no , check child in t2 and combine
# construct mape of tool 1
toolA_map = {}
with open(tool_A_out_file) as f:
    for line in f.readlines():
        line = line.strip()
        cmd, a_d = line.split("\t")
        toolA_map[cmd]= a_d
print(len(toolA_map.keys()))

49


In [160]:
# construct map of tool 2
toolB_map = {}
import os.path
from os import path

if path.isfile(tool_B_out_file):
    with open(tool_B_out_file) as f2:
        for line in f2.readlines():
            line = line.strip()
            cmd, child, child_dict = line.split("\t")
            if cmd in toolB_map and child in toolB_map[cmd]:
                print("BUGGG")
            if cmd not in toolB_map:
                toolB_map[cmd] = {}
            toolB_map[cmd][child] = child_dict        
print(len(toolB_map.keys()))

13


In [161]:
import ast
def all_yes(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    for k, val in a_dict.items():
        if type(val) == list and val[0] == "no":
            return False
    return True

def clean_dict_1(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    new_d = {}
    for k, val in a_dict.items():
        if type(val) == list:
            if val[0] in ["yes", "no"]:
                new_d[k] = val[1]                
        elif type(val) == dict:
            new_d[k] = a_dict(val[1])
        else:
            new_d[k] = val
    # only for now
    if 'dance_type_span' in new_d:
        new_d['dance_type'] = {}
        new_d['dance_type']['dance_type_name'] = new_d['dance_type_span']
        new_d.pop('dance_type_span')
    if 'dance_type_name' in new_d:
        new_d['dance_type'] = {}
        new_d['dance_type']['dance_type_name'] = new_d['dance_type_name']
        new_d.pop('dance_type_name')
    return new_d 

In [162]:
# post_process spans and "contains_coreference" : "no"
def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        a = min(a, indices[i][0])
        b = max(b, indices[i][1])
    return [a, b]

def fix_put_mem(d):
    
    if type(d) == str:
        d = ast.literal_eval(d)
    new_d = copy.deepcopy(d)
    del new_d['action_type']
    if 'has_tag' in new_d and 'upsert' in new_d:
        new_d['upsert']['memory_data']['has_tag'] = new_d['has_tag']
        del new_d['has_tag']

    return new_d

def fix_spans(d):
    new_d = {}
    if type(d) == str:
        d = ast.literal_eval(d)
    for k, v in d.items():
        if k == "contains_coreference" and v == "no":
            continue
        if type(v) == list:
            if k == "tag_val":
                new_d["has_tag"] = [0, merge_indices(v)] 
            else:
                new_d[k] = [0, merge_indices(v)] 
            continue
        elif type(v) == dict:
            new_d[k] = fix_spans(v)
            continue
        else:
            new_d[k] = v
    return new_d

In [163]:
# combine and write output to a file
i  =0
# what these action will look like in the map
import json
import copy
dance_type_map = {
    'point': 'point',
    'look': 'look_turn',
    'turn' : 'body_turn'
}

from pprint import pprint
# update dict of tool1 with tool 2
folder_name_combined = '/Users/kavyasrinet/Downloads/combined_tools'
#folder_name_combined = '/Users/kavyasrinet/Downloads/'
# with open(folder_name_combined + 'all_combined.txt', 'w') as f:
with open(parent_folder + '/all_combined.txt', 'w') as f:
    for cmd, a_dict in toolA_map.items():
        # remove the ['yes', val] etc
        clean_dict = clean_dict_1(a_dict)
        print(clean_dict)
        if all_yes(a_dict):
            action_type = clean_dict['action_type']

            valid_dict = {}
            valid_dict['dialogue_type'] = clean_dict['dialogue_type']
            del clean_dict['dialogue_type']
            clean_dict['action_type'] = clean_dict['action_type'].upper()
            act_dict = fix_spans(clean_dict)
            valid_dict['action_sequence'] = [act_dict]
        
            f.write(cmd + "\t" + json.dumps(valid_dict) + "\n")
            print(cmd)
            print(valid_dict)
            print("All yes")
            print("*"*20)
            continue
        if clean_dict['action_type'] == 'noop':
            f.write(cmd + "\t" + json.dumps(clean_dict) + "\n")
            print(clean_dict)
            print("NOOP")
            print("*"*20)
            continue
        if clean_dict['action_type'] == 'otheraction':
            f.write(cmd + "\t" + str(a_dict) + "\n")
            continue

        

        if toolB_map and cmd in toolB_map:
            #print(cmd)
            child_dict_all = toolB_map[cmd]
            # update action dict with all children except for reference object
            for k, v in child_dict_all.items():
                if k not in clean_dict:
                    print("BUGGGG")
                if type(v) == str:
                    v = ast.literal_eval(v)
                #print(k, v)
                if not v:
                    continue 

                if 'reference_object' in v[k]:
                    #print("HHHH")
                    value = v[k]['reference_object']
                    v[k]['reference_object'] =  fix_ref_obj(value)
    #                 new_clean_dict = {}
    #                 up_dict = {}
    #                 if 'special_reference' in val:
    #                     new_clean_dict['special_reference'] = val['special_reference']
    #                     val.pop('special_reference')
    #                 if 'repeat' in val:
    #                     new_clean_dict['repeat'] = val['repeat']
    #                     val.pop('repeat')
    #                 if val:
    #                     new_clean_dict['filters'] = val
    #                 up_dict['reference_object'] = new_clean_dict
    #                 v = {k: up_dict}
                #print(cmd, a_dict, child_dict)
                if k == "tag_val":
                    clean_dict.update(v)
                elif k == "facing":
                    action_type = clean_dict['action_type']
                    # set to dance
                    clean_dict['action_type'] = 'DANCE'
                    clean_dict['dance_type'] = {dance_type_map[action_type]: v['facing']}
                    clean_dict.pop('facing')
                else:
                    clean_dict[k] = v[k]
    #     print("after tool B")
    #     pprint(clean_dict)
    #     print("after tool C")
    #     # now add reference object dict
    #     pprint(clean_dict)
        ref_obj_dict = {}
        if toolC_updated_map and cmd in toolC_updated_map:
            ref_obj_dict = toolC_updated_map[cmd]
        clean_dict.update(ref_obj_dict)
        if 'receiver_reference_object' in clean_dict:
            clean_dict['receiver'] = {'reference_object': clean_dict['receiver_reference_object']}
            clean_dict.pop('receiver_reference_object')
        if 'receiver_location' in clean_dict:
            clean_dict['receiver'] = {'location': clean_dict['receiver_location']}
            clean_dict.pop('receiver_location')

        actual_dict = copy.deepcopy((clean_dict))

        action_type = actual_dict['action_type']

        valid_dict = {}
        valid_dict['dialogue_type'] = actual_dict['dialogue_type']
        del actual_dict['dialogue_type']
        actual_dict['action_type'] = actual_dict['action_type'].upper()
        act_dict = fix_spans(actual_dict)
        valid_dict['action_sequence'] = [act_dict]
        print(cmd)
        pprint(valid_dict)
        print("*"*40)
        f.write(cmd + "\t" + json.dumps(valid_dict) + "\n")

{'action_type': 'get', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'reference_object': [[2, 2]], 'receiver_location': [[5, 5]]}
and add it to the tree
{'action_sequence': [{'action_type': 'GET',
                      'receiver': {'location': {'reference_object': {'filters': {'has_name': [0,
                                                                                              [5,
                                                                                               5]]}}}},
                      'reference_object': {'filters': {'contains_coreference': 'yes'}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
****************************************
{'action_type': 'get', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'reference_object': [[3, 3], [4, 4], [5, 5], [6, 6]], 'receiver_reference_object': [[2, 2]]}
bot bring me my red pants suit
{'action_sequence': [{'action_type': 'GET',
                      'receiver': {'reference_object': {'special_reference': 'SPEAKER'}},
                    

In [232]:
with open('/Users/kavyasrinet/Github/annotated_data/with_get_give_bring/annotated_data_combined.txt') as f:
    print(len(f.readlines()))

43


In [8]:
# for composites
# now skip composite_action and separate otheraction from combined dicts
mypath = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/composites/'
mypath = '/Users/kavyasrinet/Downloads/'
import ast
import json

f = []
all_cnt  = 0
cmp  = 0
other = 0
valid = 0
all_comp = set()
with open(mypath + "all_combined.txt") as f_read, open(mypath + 'all_combined_final.txt', 'w') as f:
    for line in f_read.readlines():
        line = line.strip()

        text, d = line.split("\t")
        actual_dict = ast.literal_eval(d.strip())
        action_type = actual_dict['action_type']
        if action_type == 'composite_action':
            print(line)
            cmp += 1
            all_comp.add(text.strip())
            continue
        else:
            valid += 1
            valid_dict = {}
            valid_dict['dialogue_type'] = actual_dict['dialogue_type']
            del actual_dict['dialogue_type']
            actual_dict['action_type'] = actual_dict['action_type'].upper()
            valid_dict['action_sequence'] = [actual_dict]
            f.write(text.strip() + "\t" + json.dumps(valid_dict) + "\n")
print(cmp)
print(len(all_comp))
print(other)
print(valid)

0
0
0
5


In [548]:
# now skip composite_action and separate otheraction from combined dicts
mypath = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/composites/'
from os import walk
import ast
import json

f = []
all_cnt  = 0
cmp  = 0
other = 0
valid = 0
all_comp = set()
with open(mypath + "all_final_annotations.txt", 'w') as f_final, open(mypath + "all_other_actions.txt", 'w') as f_other:
    for (dirpath, dirnames, filenames) in walk(mypath):
        if filenames:
            for file_name in filenames:
                if not (file_name.startswith('.') or file_name in ['all_final_annotations.txt', 'all_other_actions.txt']):
                    fn  = dirpath + file_name
                    with open(fn) as f:
                        #print(fn, len(f.readlines()))
                        #all_cnt += len(f.readlines())
                        for line in f.readlines():
                            #print(line)
                            line = line.strip()

                            text, d = line.split("\t")
                            actual_dict = ast.literal_eval(d.strip())
                            #print(f)
                            action_type = actual_dict['action_type']
                            #print(action_type)
                            #print(action_type)
                            if action_type == 'composite_action':
                                cmp += 1
                                all_comp.add(text.strip())
                                continue
                            elif action_type[1] == 'otheraction':
                                other += 1
                                f_other.write(line + "\n")
                            else:
                                valid += 1
                                valid_dict = {}
                                valid_dict['dialogue_type'] = actual_dict['dialogue_type']
                                del actual_dict['dialogue_type']
                                #print(actual_dict['action_type'])
                                actual_dict['action_type'] = actual_dict['action_type'].upper()
                                valid_dict['action_sequence'] = [actual_dict]
                                f_final.write(text.strip() + "\t" + json.dumps(valid_dict) + "\n")
print(cmp)
print(len(all_comp))
print(other)
print(valid)

9
9
259
1893


In [647]:

        
i = 0
input_file = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/composites/all_combined_final.txt'
output_file = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/composites/all_combined_final_postprocessed.txt'
with open(input_file) as f, open(output_file, 'w') as f_w:
    for line in f.readlines():
        i+= 1
        line = line.strip()
        text, d = line.split("\t")
        
        d = ast.literal_eval(d)
#         if text.split()[0] in ['put', 'place', 'install']:
#             print(text, d['action_sequence'][0])
        action_dict = fix_spans(d['action_sequence'][0])
        if action_dict['action_type'] == 'TAG':
            updated_dict = fix_put_mem(action_dict)
            new_d = {}
            new_d['dialogue_type'] = d['dialogue_type']
            new_d.update(updated_dict)
        elif action_dict['action_type'] == 'ANSWER':
            #print(d)
            new_d = {}
            new_d['dialogue_type'] = 'GET_MEMORY'
        else:
            if action_dict['action_type'] == 'COPY':
                action_dict['action_type'] = 'BUILD'
            d['action_sequence'] = [action_dict]
            new_d = d
        f_w.write(text+  "\t" + json.dumps(new_d) + "\n")
        

In [173]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

    
def find_int(sentence):
    words= sentence.split()
    for word in words:
        if RepresentsInt(word):
            return True
    return False
    
import random
def fix_int(sentence):
    random_degrees = ['-90', '-45', '45', '90', '135', '-135', '180', '-180', '360', '-360']
    words = sentence.strip().split()
    new_words= []
    for word in words:
        if RepresentsInt(word) :#and 'degrees' in words:
            random_num = random.randrange(-360, 360)
            new_words.append(str(random_num))
        else:
            new_words.append(word)
    
    return " ".join(new_words).strip()

In [1]:
import json

orig_data = {}
with open('/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/first_65/all_combined.txt') as f:
    for line in f.readlines():
        line  = line.strip()
        t, d  =line.split("\t")
        d = json.loads(d)
        orig_data[t] = d
print(len(orig_data.keys()))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/first_65/all_combined.txt'

In [318]:
more_num_data = {}
for k, v in orig_data.items():
    if find_int(k):
        for i in range(50):
            new_t = fix_int(k)
            more_num_data[new_t] = v
        
print(len(more_num_data.keys()))     

532


In [363]:
dirs_map = {
    "left": "-90",
    "west": "-90",
    "right": "90",
    "east": "90",
    "front" : "0",
    "north": "0",
    "back": "180",
    "south": "180",
    "front left": "-45",
    "northwest": "-45",
    "northeast": "45",
    "front right": "45",
    "southwest": "-135",
    "southeast": "135",
    "all the way around clockwise": "360",
    "all the way around anticlockwise": "-360",
    "all the way around": "360"
}

vertical_dirs_map = {
    "up": "90",
    "down": "-90"
}

import copy
print(len(dirs_map.keys()))

def substitute_dirs(k, v, key_name='relative_yaw', alternate=False):
    words = k.split()
    new_words = []
    new_d = copy.deepcopy(v)
#     print(k)
#     print(v)
#     return{}
    
    # find other keys that are not this direction key
    other_keys = []
    for x in words:
        if (x in dirs_map) or (x in vertical_dirs_map):
            if (key_name == 'relative_yaw' and alternate == False) or (key_name == 'relative_pitch' and alternate==True):
                new_d = copy.deepcopy(v)
                k2 = list(new_d['dance_type'].keys())[0]
                if (key_name in new_d['dance_type'][k2]) and (not new_d['dance_type'][k2][key_name].get('angle', None)):
                    return {k : v}
                if key_name not in new_d['dance_type'][k2]:
                    return {k : v}
                this_dir = x
                other_keys = list(dirs_map.keys())
                if this_dir in other_keys:
                    other_keys.remove(this_dir)
            elif (key_name == 'relative_yaw' and alternate == True) or (key_name == 'relative_pitch' and alternate==False):
        
                new_d = copy.deepcopy(v)
                k2 = list(new_d['dance_type'].keys())[0]
                if (key_name in new_d['dance_type'][k2]) and (not new_d['dance_type'][k2][key_name].get('angle', None)):
                    return {k : v}
                if key_name not in new_d['dance_type'][k2]:
                    return {k : v}
                this_dir = x
                other_keys = list(vertical_dirs_map.keys())
                if this_dir in other_keys:
                    other_keys.remove(this_dir)

    new_dict = {}
    for k in other_keys:
        new_words = []
        new_d = copy.deepcopy(v)
        
        for x in words:
            # if direction word, pick new
            if (x in dirs_map) or (x in vertical_dirs_map):
                # it has just one key
                k2 = list(new_d['dance_type'].keys())[0]
                if (key_name in new_d['dance_type'][k2]) and (not new_d['dance_type'][k2][key_name].get('angle', None)):
                    print("BUGGGGG")
                new_words.append(k)
                if alternate == False:
                    if key_name == 'relative_yaw':
                        new_d['dance_type'][k2][key_name]['angle'] = dirs_map[k]
                    else:
                        new_d['dance_type'][k2][key_name]['angle'] = vertical_dirs_map[k]
                elif alternate == True:
                    if key_name == 'relative_yaw':
                        new_d['dance_type'][k2]['relative_pitch'] = {}
                        new_d['dance_type'][k2]['relative_pitch']['angle'] = vertical_dirs_map[k]
                        new_d['dance_type'][k2].pop('relative_yaw')
                    elif key_name == 'relative_pitch':
                        new_d['dance_type'][k2]['relative_yaw'] = {}
                        new_d['dance_type'][k2]['relative_yaw']['angle'] = dirs_map[k]
                        new_d['dance_type'][k2].pop('relative_pitch')
                        
            else:
                new_words.append(x)
        new_dict[" ".join(new_words)] = new_d
    return new_dict
                

17


In [364]:
more_rel_dir_data = {}
all_dir_keys = set(dirs_map.keys())
all_dir_keys.update(set(vertical_dirs_map.keys()))
print(all_dir_keys)
print("*"*20)
for k, v in orig_data.items():
    words = k.split()
    
    if any(x in words for x in all_dir_keys):
#         print(k, v)
#         print("*"*20)
        
        #this_dict = substitute_dirs(k, v, key_name='relative_yaw', alternate=False)
        #this_dict = substitute_dirs(k, v, key_name='relative_yaw', alternate=True)
        this_dict = substitute_dirs(k, v, key_name='relative_pitch', alternate=False)
#         this_dict = substitute_dirs(k, v, key_name='relative_pitch', alternate=True)
        more_rel_dir_data.update(this_dict)
        print(k, v)
        print(this_dict)
        print("-"*20)
    
#         print(len(more_rel_dir_data.keys()))
#         print(more_rel_dir_data)
#         print(k, v)
#         break
        #new_k, new_v = substitute_dirs(k, v)
#print(len(more_rel_dir_data.keys()))


{'front right', 'northeast', 'all the way around anticlockwise', 'up', 'front', 'left', 'front left', 'right', 'southwest', 'west', 'north', 'south', 'all the way around clockwise', 'northwest', 'back', 'southeast', 'all the way around', 'east', 'down'}
********************
aim your head 10 degrees up {'action_type': 'DANCE', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'dance_type': {'look_turn': {'relative_pitch': {'pitch_span': [[3, 3], [4, 4], [5, 5]]}}}}
{'aim your head 10 degrees up': {'action_type': 'DANCE', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'dance_type': {'look_turn': {'relative_pitch': {'pitch_span': [[3, 3], [4, 4], [5, 5]]}}}}}
--------------------
face left {'action_type': 'DANCE', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'dance_type': {'look_turn': {'relative_yaw': {'angle': '-90'}}}}
{'face left': {'action_type': 'DANCE', 'dialogue_type': 'HUMAN_GIVE_COMMAND', 'dance_type': {'look_turn': {'relative_yaw': {'angle': '-90'}}}}}
--------------------
face west {'action_type': 'DA

In [275]:
orig_data.update(more_num_data)
print(len(orig_data.keys()))

599


In [276]:
orig_data.update(more_rel_dir_data)
print(len(orig_data.keys()))

691


In [277]:
new_postprocessed_data = {}
for k, v in orig_data.items():
    actual_dict = copy.deepcopy((v))
    
    action_type = actual_dict['action_type']

    valid_dict = {}
    valid_dict['dialogue_type'] = actual_dict['dialogue_type']
    del actual_dict['dialogue_type']
    actual_dict['action_type'] = actual_dict['action_type'].upper()
    act_dict = fix_spans(actual_dict)
    valid_dict['action_sequence'] = [act_dict]
    
    new_postprocessed_data[k] = valid_dict
print(len(new_postprocessed_data.keys()))

691


In [278]:
with open('/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/first_65/augmented_combined.json', 'w') as f:
    json.dump(new_postprocessed_data, f)

In [228]:
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/first_65/augmented_combined.json') as f:
    data = json.load(f)
    

In [279]:
gt_new = {}
i = 0
for k, v in new_postprocessed_data.items():
    if i == 500:
        break
    else:
        i += 1
        gt_new[k] = v
print(len(gt_new.keys()))

500


In [315]:
# look + up etc , + pitch 
# look right + yaw
for k, v in new_postprocessed_data.items():
    if 'turn' in k:
        print(k)
        pprint(v)
        print("*"*20)

turn -25 degrees
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                                 [1,
                                                                                  2]]}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn 30 degrees left
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                                 [1,
                                                                                  3]]}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn 30 degrees to your left
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                   

{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                                 [1,
                                                                                  3]]}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn -156 degrees left
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                                 [1,
                                                                                  3]]}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn -183 degrees left
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'yaw_span': [0,
                                                                                 [1,
   

                      'dance_type': {'body_turn': {'yaw': [0, [2, 3]]}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn by 74 degrees
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'yaw': [0, [2, 3]]}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn by -118 degrees
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'yaw': [0, [2, 3]]}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn by 95 degrees
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'yaw': [0, [2, 3]]}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn by 17 degrees
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'yaw': [0, [2, 3]]}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn by -247 degrees
{'action_sequence': [{'action_type': 

                      'dance_type': {'body_turn': {'relative_yaw': {'angle': '135'}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn to my all the way around clockwise
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'angle': '360'}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn to my all the way around antiuclockwise
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'angle': '-360'}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn to my all the way around
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'angle': '360'}}}}],
 'dialogue_type': 'HUMAN_GIVE_COMMAND'}
********************
turn to your west
{'action_sequence': [{'action_type': 'DANCE',
                      'dance_type': {'body_turn': {'relative_yaw': {'angle': '-90'

In [282]:
gt_data = {}
with open('../../minecraft/python/craftassist/ground_truth_data.txt') as f:
    for line in f.readlines():
        text, ad = line.strip().split("\t")
        ad_new = json.loads(ad)
        if type(ad_new) == str:
            ad_new = json.loads(ad_new)
        gt_data[text] = ad_new
print(len(gt_data.keys()))

431760


In [300]:
all_actions = set()
for k, v in gt_data.items():
    if v['dialogue_type'] == 'HUMAN_GIVE_COMMAND' and 'action_sequence' in v and len(v['action_sequence']) == 1:
        new_dict = copy.deepcopy(v)
        act = new_dict['action_sequence'][0]
        all_actions.add(act['action_type'])
        
#         if act['action_type'] == 'DANCE':
#             if 'dance_type' in act and type(act['dance_type']) == dict:
#                 key = list(act['dance_type'].keys())[0]
#                 if 'relative_yaw' in act['dance_type'][key] and 'yaw' in act['dance_type'][key]:
#                     act['dance_type'][key].pop('relative_yaw')
#                     print(k,v )
#                     print(k, act)
#                     print("*"*20)
#                     #act['dance_type'][key].pop('relative_yaw')
print(all_actions)

{'MOVE', 'RESUME', 'BUILD', 'STOP', 'SPAWN', 'DESTROY', 'FILL', 'UNDO', 'DANCE', 'FREEBUILD', 'DIG'}


In [281]:
print(len(gt_data.keys()))
gt_data.update(gt_new)
print(len(gt_data.keys()))

431760
432171


In [258]:
with open('../../minecraft/python/craftassist/ground_truth_data.txt', 'w') as f:
    for k, v in gt_data.items():
        f.write(k + "\t" + json.dumps(v)+ "\n")

In [263]:
#print(gt_new)
if 'turn by 14 degrees' in gt_new:
    print(gt_new['turn by 14 degrees'])

{'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action_sequence': [{'action_type': 'DANCE', 'dance_type': {'body_turn': {'relative_yaw': {'angle': '90'}, 'yaw': [0, [2, 3]]}}}]}


In [260]:
with open('../../minecraft/python/craftassist/ground_truth_data.txt') as f:
    for line in f.readlines():
        t , a = line.strip().split("\t")
        a = json.loads(a)
        if type(a) != dict:
            print("nooooo")